In [ ]:
!pip install stable-baselines3[extra]

In [2]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)

When taking random moves either left or right

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:29.0
Episode:2 Score:34.0
Episode:3 Score:38.0
Episode:4 Score:22.0
Episode:5 Score:15.0


In [5]:
env.action_space.sample()

0

In [6]:
env.observation_space.sample()

array([-3.4674082e+00,  6.5547698e+37,  1.3368501e-01,  2.8951397e+38],
      dtype=float32)

In [7]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


# **Training**

In [8]:
model.learn(total_timesteps=20000) # trains a model -> 1 single line

-----------------------------
| time/              |      |
|    fps             | 1398 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1088        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008951815 |
|    clip_fraction        | 0.0896      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00676    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.33        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0137     |
|    value_loss           | 48.8        |
-----------------------------------------
----------------------------------

In [9]:
# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, done, info = env.step(action)
#     # env.render()
#     if done: 
#         print('info', info)
#         break

In [10]:
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(200.0, 0.0)

# **Testing**

In [11]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:200.0
Episode:2 Score:200.0
Episode:3 Score:200.0
Episode:4 Score:200.0
Episode:5 Score:200.0


In [12]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(200.0, 0.0)

### Using a callback function

In [14]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, eval_freq=10000, verbose=1)

In [15]:
model = PPO('MlpPolicy', env, verbose = 1)
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1498 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1170        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009045578 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0107     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.88        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0179     |
|    value_loss           | 51.3        |
-----------------------------------------
-----------------

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=10000, episode_reward=180.00 +/- 17.75
Episode length: 180.00 +/- 17.75
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 180          |
|    mean_reward          | 180          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0070840144 |
|    clip_fraction        | 0.0564       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.61        |
|    explained_variance   | 0.303        |
|    learning_rate        | 0.0003       |
|    loss                 | 22.4         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0147      |
|    value_loss           | 60.3         |
------------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 960   |
|    iterations     

In [16]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(200.0, 0.0)

### Changing Policies

In [17]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1403 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 900          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0154934395 |
|    clip_fraction        | 0.218        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.682       |
|    explained_variance   | -0.00445     |
|    learning_rate        | 0.0003       |
|    loss                 | 3.83         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0222      |
|    value_loss           | 18.7         |
------------------------------------------

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0133928675 |
|    clip_fraction        | 0.12         |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.576       |
|    explained_variance   | 0.544        |
|    learning_rate        | 0.0003       |
|    loss                 | 11           |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0159      |
|    value_loss           | 35.5         |
------------------------------------------
------------------------------
| time/              |       |
|    fps             | 718   |
|    iterations      | 5     |
|    time_ela

# Using an Alternate Algorithm

In [18]:
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env, verbose = 1)
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.967    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7348     |
|    time_elapsed     | 0        |
|    total_timesteps  | 69       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.94     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6223     |
|    time_elapsed     | 0        |
|    total_timesteps  | 127      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.893    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 6119     |
|    time_elapsed     | 0        |
|    total_timesteps  | 225      |
----------------------------------
----------------------------------
| r

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 488      |
|    fps              | 5240     |
|    time_elapsed     | 2        |
|    total_timesteps  | 10519    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 492      |
|    fps              | 5239     |
|    time_elapsed     | 2        |
|    total_timesteps  | 10587    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 496      |
|    fps              | 5242     |
|    time_elapsed     | 2        |
|    total_timesteps  | 10676    |
----------------------------------
----------------------------------
| rollout/          

In [19]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(9.2, 0.7483314773547883)